##### Convolution Neural Network  
Convolution Neural Networks are a type of Deep Learning model that are specifiucally designed for Images and Videos.  
Uses:  Image Classification, Object Detection etc  
It has ability to learn spatial hieraarchies of features.

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings("ignore")

In [19]:
# Define a transformation to convert images to PyTorch tensors
transform = transforms.ToTensor()

# Load the Fashion MNIST training dataset
train_data = datasets.FashionMNIST(
    root='./data',  # Directory to store the dataset
    train=True,     # Specify this is the training set
    download=True,  # Download the dataset if not already present
    transform=transform
)

# Load the Fashion MNIST test dataset
test_data = datasets.FashionMNIST(
    root='./data',  # Directory to store the dataset
    train=False,    # Specify this is the test set
    download=True,  # Download the dataset if not already present
    transform=transform
)


In [20]:
train_dataloader = DataLoader(train_data, batch_size=256, shuffle=True, num_workers=0)
test_dataloader = DataLoader(test_data, batch_size=256, shuffle=False)


In [21]:
train_data[0][0].shape

torch.Size([1, 28, 28])

In [24]:
class my_cnn_model(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=num_features, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.LazyLinear(128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10),
        )
    
    def forward(self,x):
        x = self.features(x)
        x = self.classifier(x)
        return x

device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
num_features = train_data[0][0].shape[0]
model = my_cnn_model(num_features)
model.to(device)



my_cnn_model(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ReLU()
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): ReLU()
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): LazyLinear(in_features=0, out_features=128, bias=True)
    (2): ReLU()
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): ReLU()
    (5): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [25]:
epochs = 100
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(epochs):
    model.train()
    for batch_features, batch_labels in train_dataloader:
        batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
        y_pred = model(batch_features)
        loss = loss_fn(y_pred, batch_labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if epoch%2 == 0:
        print(f"Loss after epoch {epoch + 1}: {loss.item()}")

Loss after epoch 1: 0.2140083909034729
Loss after epoch 3: 0.2608056366443634
Loss after epoch 5: 0.1749967783689499
Loss after epoch 7: 0.12747584283351898
Loss after epoch 9: 0.21175575256347656
Loss after epoch 11: 0.15755312144756317
Loss after epoch 13: 0.1633957028388977
Loss after epoch 15: 0.018246613442897797
Loss after epoch 17: 0.06495492905378342
Loss after epoch 19: 0.07409251481294632
Loss after epoch 21: 0.0693206787109375
Loss after epoch 23: 0.057302992790937424
Loss after epoch 25: 0.010069935582578182
Loss after epoch 27: 0.010885455645620823
Loss after epoch 29: 0.10230210423469543
Loss after epoch 31: 0.08756810426712036
Loss after epoch 33: 0.002805870957672596
Loss after epoch 35: 0.012920464389026165
Loss after epoch 37: 0.04106512665748596
Loss after epoch 39: 0.0031726250890642405
Loss after epoch 41: 0.009272154420614243
Loss after epoch 43: 0.0027967102359980345
Loss after epoch 45: 0.13214771449565887
Loss after epoch 47: 0.0989958643913269
Loss after epoch

In [26]:
model.eval()
accuracy_list = []
for batch_features, batch_labels in test_dataloader:
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
    y_pred = model(batch_features)
    _, predicted = torch.max(y_pred, 1)
    accuracy = accuracy_score(batch_labels.cpu(), predicted.cpu())
    accuracy_list.append(accuracy)

average_accuracy = sum(accuracy_list) / len(accuracy_list)
print(f"Average Test Accuracy: {average_accuracy:.4f}")

Average Test Accuracy: 0.9159


In [27]:
accuracy_list = []
for batch_features, batch_labels in train_dataloader:
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
    y_pred = model(batch_features)
    _, predicted = torch.max(y_pred, 1)
    accuracy = accuracy_score(batch_labels.cpu(), predicted.cpu())
    accuracy_list.append(accuracy)

average_accuracy = sum(accuracy_list) / len(accuracy_list)
print(f"Average Test Accuracy: {average_accuracy:.4f}")

Average Test Accuracy: 0.9969
